In [1]:
import marvin
marvin.config.download = True
marvin.config.release = 'DR17'

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.stats as sci

from marvin.tools.vacs import VACs
from astropy.io import fits
import numpy as np
from marvin.tools import Maps
from marvin.tools import Cube

from marvin import config #Needed to fix "Cube not subscriptable" when running in Sciserver
config.forceDbOff() #Needed to fix "Cube not subscriptable" when running in Sciserver

print('Marvin and Matplotlib imported. Warnings about missing paths/files are typical, so no need to be concerned')

/home/idies/miniconda3/envs/py38/lib/python3.8/site-packages/marvin/core/exceptions.py:50: UserWarning: cannot initiate Sentry error reporting: [Errno 6] No such device or address.
  warnings.warn('cannot initiate Sentry error reporting: {0}.'.format(str(ee)),
[INFO]: No release version set. Setting default to DR17
[WARNING]: path /home/idies/workspace/sdss_sas/mangawork/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/idies/workspace/sdss_sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


Marvin and Matplotlib imported. Warnings about missing paths/files are typical, so no need to be concerned


In [2]:
selection = 'full_initial_sample.fits'
hdu = fits.open(selection)
data = hdu[1].data

mangaplifu = []

for galaxy in range(len(data)):
    if data[galaxy]['GZ_SPIRAL_VOTES']/data[galaxy]['GZ_TOTAL_CLASSIFICATIONS']>0.2 and data[galaxy]['IFUDESIGNSIZE_1']>0 and data[galaxy]['nsa_elpetro_ba']>0.5:
        mangaplifu.append(str(data[galaxy]['plateifu_1']))
        
hdu.close()
#selecting galaxies with spiral arms using GZ3D metadata

In [2]:
#a function to get appropriate values for arm vs interarm spaxels
def arm_selector (mangaplifu):
    plate = mangaplifu.split('-')[0]
    ifu = mangaplifu.split('-')[1]
    
    pi3D = 'https://data.sdss.org/sas/dr17/manga/spectro/pipe3d/v3_1_1/3.1.1/'+str(plate)+'/manga-'+str(plate)+'-'+str(ifu)+'.Pipe3D.cube.fits.gz'
    #Getting the FITS files for Pipe3D
    hdu1 = fits.open(pi3D)
    mass = hdu1[1].data[19]
    
    #Getting the GZ3D files
    maps = marvin.tools.Maps(mangaplifu,mode='remote')
    data = maps.vacs.gz3d
    
    #selecting arm vs interarm spaxels
    galaxy = data.spiral_mask_spaxel
    at=5 #arm-interarm thresholds
    it=3
    arm = np.where(galaxy > at,1,np.nan)
    interarm = np.where(galaxy < it,1,np.nan)
    ma = mass*arm
    mi= mass*interarm
    
    #getting binned averages
    radius = maps.spx_ellcoo_r_re
    mask=np.where(np.isnan(ma),0,1)
    mask2=np.where(np.isnan(mi),0,1)
    bins=25
    drange=[0,3.0]
    sigmass_ma,radius_ma,binnumber = sci.binned_statistic(radius.value[mask!=0],ma[mask!=0],bins=bins,range=drange,statistic='std')
    count_ma,radius_ma,binnumber = sci.binned_statistic(radius.value[mask!=0],ma[mask!=0],bins=bins,range=drange,statistic='count')
    mass_ma,radius_ma,binnumber = sci.binned_statistic(radius.value[mask!=0],ma[mask!=0],bins=bins,range=drange,statistic='mean')
    mass_mi,radius_mi,binnnumber = sci.binned_statistic(radius.value[mask2!=0],mi[mask2!=0],bins=bins,range=drange,statistic='mean')
    sigmass_mi,radius_mi,binnumber = sci.binned_statistic(radius.value[mask2!=0],mi[mask2!=0],bins=bins,range=drange,statistic='std')
    count_mi,radius_mi,binnumber = sci.binned_statistic(radius.value[mask2!=0],mi[mask2!=0],bins=bins,range=drange,statistic='count')
    
    hdu1.close()
    
    return mass_ma,radius_ma,mass_mi,radius_mi,sigmass_ma,sigmass_mi,count_ma,count_mi

In [3]:
mangaplifu=['11831-3701', '11018-12702', '11830-12702', '9879-9101', '8321-12704', '8321-12705', '10515-12704', '10845-12704', '10516-12704', '10519-12705', '10520-6102', '10520-6104', '11866-12701', '12506-12701', '11022-12703', '8589-12704', '8553-9102', '8553-12704', '8481-9102', '8481-12701', '8546-12704', '8546-6104', '8481-1902', '8561-12701', '8561-9101', '8481-6102', '8481-12704', '8563-1901', '8563-3701', '8561-9102', '8548-6103', '8482-1902', '8482-9101', '8482-9102', '8563-12705', '8562-12704', '8562-3702', '8484-12702', '8548-6102', '8484-6103', '8548-12702', '8549-6104', '8549-12703', '8548-12703', '8482-12703', '8483-6103', '8484-3701', '8484-3703', '8484-1902', '8484-6104', '8978-1901', '9026-12702', '9026-3703', '9026-9101', '11945-12702', '11945-6101', '8588-12703', '11945-12705', '11945-6104', '8606-12701', '8612-9101', '8612-6104', '8612-12702', '9187-9101', '11982-9102', '9187-3701', '9187-12702', '9187-12705', '12678-9102', '12678-12705', '12679-6104', '12679-12705', '8625-12705', '8611-12702', '8611-6103', '11025-6102', '8625-6104', '8626-1902', '11865-6101', '8624-9101', '8624-9102', '8945-12705', '8624-12703', '8259-9102', '8259-3702', '8259-12704', '8260-9101', '8261-6103', '11755-12703', '8263-12705', '10221-12705', '10221-6102', '10221-6104', '9508-3704', '9508-12701', '9508-3703', '8939-12703', '9508-12704', '9508-6104', '9507-12702', '11747-6101', '10499-3702', '10499-12705', '12488-12705', '12488-12701', '9506-12704', '12488-12704', '12488-9102', '10501-6102', '10501-1902', '10500-3702', '10500-1901', '10500-6104', '10505-9101', '11749-12702', '10505-12704', '10505-12703','9502-6101', '9502-6103', '9502-12702', '10498-12702', '9493-12704', '8155-12701', '9501-9102', '9500-6103', '9506-12701', '9506-6101', '9506-3702', '9506-9101', '9500-9102', '8987-3703', '8987-12702', '12488-3703', '12487-12705', '8155-12704', '9505-3702', '11748-12701', '9505-12704', '8156-12702', '10838-9102', '10838-12701', '8944-1901', '8944-3703', '8944-9102', '8150-9102', '8569-12701', '8943-12702', '8943-12701', '8568-3704', '8568-12705', '12483-12704', '8943-12703', '8982-12703', '8984-12704', '12624-12702', '8091-9101', '12092-12704', '8988-12704', '11750-9101', '11750-9102', '8988-12705', '8988-3703', '8091-6101', '8988-9102', '8091-12704', '11750-12701', '12484-9101', '8554-3704', '8988-12701', '9879-6101', '8322-6102', '8322-6101', '8981-6101', '8981-3702', '12093-12705', '8330-12702', '8330-1902', '8934-12701', '11826-12704', '9875-12704', '12483-6103', '12483-12705', '8336-6101', '8952-6104', '12621-12705', '12093-9101', '8445-12703', '8442-6103', '8093-6104', '8445-6104', '8446-6104', '8093-9102', '11743-12703', '10217-6103', '10216-6101', '10218-12701', '10218-9101', '10218-3704', '9488-12705', '9493-1902', '9493-6101', '9493-9102', '8093-12705', '9505-3701', '8570-1901', '8323-6101', '8095-12701', '12621-6103', '12624-12704', '8095-3702', '8157-12702', '8446-12705', '9513-12704', '8330-12701', '12622-9101', '12620-6101', '8442-12704', '8157-12703', '12622-1901', '8322-9102', '8442-6101', '8554-12701', '8554-12702', '9513-3703', '8985-6102', '12624-12703', '12622-1902', '8985-12703','8152-12701','8152-9102','8462-6101','8462-1902','12769-12701', '8656-6102', '8656-6103', '8337-6101', '8337-12703', '8656-1902', '12769-12703', '9024-12705', '8656-12703', '9002-12704', '8656-1901', '9045-1902', '9045-12703', '9045-12704', '8657-1902', '8657-6102', '8657-9101', '9042-9101', '9890-6102', '9042-12703', '9888-12704', '9886-12703', '8657-3702', '9094-3701', '9089-6104', '9095-1901', '8657-6101', '9095-6102', '9095-9102', '9050-9101', '9050-3704', '11017-12705', '11017-12703', '11017-9101', '11017-12701', '11830-6104', '11017-12704', '8657-9102', '9884-12702', '11827-12703', '8326-12703', '11827-12704', '9868-6102', '9868-12705', '9868-9102', '9033-9102', '8980-6104', '8552-9101', '8552-12703', '9872-12701', '8552-3704', '9871-12704', '9872-9101', '8315-12705', '8315-6101', '11944-12701', '9027-12704', '9025-1902', '11823-3702', '11984-12704', '11984-6102', '11980-12703', '11984-12703', '11952-12701', '11952-9102', '11952-12703', '9893-12701', '12696-12704', '12696-12705', '12073-12703', '12772-12701', '11970-12702', '12772-12705', '11970-12704', '11970-6103', '11962-12703', '11970-12705', '12772-6103', '11967-12703', '12074-9101', '12074-12705', '11967-12702', '11967-9102', '11965-12705', '11963-12702', '11960-12701', '11974-12702', '12074-12703', '8135-6103', '8132-12703', '8137-12703', '8138-12701', '8138-3702', '8138-12704', '8134-9102', '8239-9101', '8138-6104', '8714-12703', '12078-9101', '8719-6103', '8085-12701', '9489-12703', '8939-12701', '9489-12705', '9494-3704', '9494-9102', '9494-12705', '9493-9101', '9493-3704', '8155-6102', '9501-6102', '9502-12703', '9501-1901']

In [4]:
error=[]

file = open('arm_vs_interarm.txt','a')

for i in mangaplifu[1:120]:
    try:
        mass_ma,radius_ma,mass_mi,radius_mi,sigmass_ma,sigmass_mi,count_ma,count_mi= arm_selector(i)
        f=((10**(mass_ma))-(10**(mass_mi)))/(10**(mass_mi))
        for j in range(len(mass_ma)):
            row=[str(i),' ', str(radius_ma[j]),' ', str(mass_ma[j]),' ', str(mass_mi[j]),' ', str(f[j]),' ',str(sigmass_ma[j]),' ',str(sigmass_mi[j]),' ',str(count_ma[j]),' ',str(count_mi[j])]
            file.writelines(row)
            file.write('\n')
        print('Successful:',i)
    except:
        error.append(i)
        print('Problems with:',i)
    continue
file.close()
print(error)
#Getting the average mass and radius values and the excess in mass values

Successful: 11018-12702
Successful: 11830-12702
Successful: 9879-9101
Successful: 8321-12704
Successful: 8321-12705
Successful: 10515-12704
Successful: 10845-12704
Successful: 10516-12704
Successful: 10519-12705
Successful: 10520-6102
Successful: 10520-6104
Successful: 11866-12701
Successful: 12506-12701
Successful: 11022-12703
Successful: 8589-12704
Successful: 8553-9102
Successful: 8553-12704
Successful: 8481-9102
Successful: 8481-12701
Successful: 8546-12704
Successful: 8546-6104
Successful: 8481-1902
Successful: 8561-12701
Successful: 8561-9101
Successful: 8481-6102
Successful: 8481-12704
Successful: 8563-1901
Successful: 8482-1902
Successful: 8482-9101
Successful: 8482-9102
Successful: 8563-12705
Successful: 8562-12704
Successful: 8562-3702
Successful: 8484-12702
Successful: 8548-6102
Successful: 8484-6103
Successful: 8548-12702
Successful: 8549-6104
Successful: 8549-12703
Successful: 8548-12703
Successful: 8482-12703
Successful: 8483-6103
Successful: 8484-3701
Successful: 8484-37

In [2]:
file=np.loadtxt('arm_vs_interarm.txt',dtype=str,unpack=True)

In [3]:
l=[]
for i in file[0]:
    for j in file[0]:
        if i==j:
            l.append(i)
print(l)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

